# CART Decision Tree

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_theme()

## Pre-processing

Load and pre-process the dataset:

In [2]:
from lib import pre_processing
file_path = "./data/features_3_sec.csv"
df = pd.read_csv(file_path)
df = pre_processing(df)

## Data Normalization

There is no need to normalize the data with decision trees.

In [3]:
df_numpy = df.to_numpy()
X = df_numpy[:,:-1]
y = df_numpy[:, -1]

In [4]:
X.shape, y.shape

((9990, 57), (9990,))

However, we need a numerical enconding for the labels.

In [5]:
df_numpy = df.to_numpy()
X = df_numpy[:,:-1]
y = df_numpy[:, -1]

In [6]:
from lib import encode_labels
y_ohe = encode_labels(y)

In [7]:
X.shape, y_ohe.shape

((9990, 57), (9990, 10))

## Model Evaluation

### Multiclass CART DT

In [8]:
from sklearn import tree
from sklearn.model_selection import train_test_split

def fit_dtree(X, y, test_size):
    # Train-validation split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0)
    
    # Fit the model
    dtree = tree.DecisionTreeClassifier()
    dtree = dtree.fit(X_train, y_train)
    y_pred = dtree.predict(X_test)
    
    # Return the accuracy
    return ((y_test == y_pred).sum() / len(y_test))

In [9]:
accuracy = fit_dtree(X, y, 0.25)
print(f"| Genre: multiclass | Accuracy: {accuracy:.3%} |")

| Genre: multiclass | Accuracy: 64.131% |


In [10]:
# from sklearn.tree import export_text
# print(export_text(dtree, feature_names=list(df_num.columns)))

### Balanced Binary CART DT

In [11]:
genres = ['blues', 'classical', 'country', 'disco', 'hiphop','jazz', 'metal', 'pop', 'reggae', 'rock']

In [12]:
from lib import balance_classes
for genre in genres:
    Xi, yi = balance_classes(X, y, genre)
    accuracy = fit_dtree(Xi, yi, 0.25)
    print(f"| Genre: {genre:9} | Accuracy: {accuracy:.3%} |")

| Genre: blues     | Accuracy: 81.800% |
| Genre: classical | Accuracy: 93.387% |
| Genre: country   | Accuracy: 80.762% |
| Genre: disco     | Accuracy: 79.400% |
| Genre: hiphop    | Accuracy: 84.369% |
| Genre: jazz      | Accuracy: 84.800% |
| Genre: metal     | Accuracy: 91.800% |
| Genre: pop       | Accuracy: 88.800% |
| Genre: reggae    | Accuracy: 83.400% |
| Genre: rock      | Accuracy: 75.150% |


### Cross-validation

In [13]:
from sklearn.model_selection import KFold

def cross_validate(X, y, n_splits):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=0)
    accuracy_list = []
    
    for train, test in kf.split(X):
        X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]

        # Fit the model and compute predictions
        dtree = tree.DecisionTreeClassifier()
        dtree = dtree.fit(X_train, y_train)
        y_pred = dtree.predict(X_test)

        accuracy = (y_test == y_pred).sum() / len(y_test)
        accuracy_list.append(accuracy)
        
    return np.mean(accuracy_list)

## Static partitioning (train-validation split)

In [14]:
test_sizes = [0.1, 0.25, 0.33]

In [15]:
from lib import plot_evaluation_results
sp_results = []

for test_size in test_sizes:
    # Multiclass classifier
    accuracy = fit_dtree(X, y, test_size)
    sp_results.append(('multiclass', accuracy*100, f'Test size {test_size}'))
    
    # Binary classifiers
    for genre in genres:
        Xi, yi = balance_classes(X, y, genre)
        accuracy = fit_dtree(Xi, yi, test_size)
        sp_results.append((genre, accuracy*100, f'Test size {test_size}'))
    
# Plot
results_df = pd.DataFrame(sp_results, columns=('label', 'accuracy', 'size'))
plot_evaluation_results(results_df, 'CART Decision Tree classification - Train-validation split')

## Dynamic partitioning (cross-validation)

In [16]:
folds = [5, 10, 20]

In [17]:
sp_results = []

for fold in folds:
    # Multiclass classifier
    accuracy = cross_validate(X, y, fold)
    sp_results.append(('multiclass', accuracy*100, f'{fold}-fold CV'))
    
    # Binary classifiers
    for genre in genres:
        Xi, yi = balance_classes(X, y, genre)
        accuracy = cross_validate(Xi, yi, fold)
        sp_results.append((genre, accuracy*100, f'{fold}-fold CV'))
    
# Plot
results_df = pd.DataFrame(sp_results, columns=('label', 'accuracy', 'size'))   
plot_evaluation_results(results_df, 'CART Decision Tree classification - Cross-validation')